In [1]:
from collections import Counter
import emoji
from io import StringIO
import math
from nltk.corpus import stopwords
import numpy as np
import os
import pandas as pd
from pprint import pprint
from random import randint
import re
from scipy.sparse import csr_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC, SVC
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (accuracy_score, f1_score, matthews_corrcoef, precision_score, 
                             precision_recall_fscore_support, recall_score, roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
import statistics as st

In [2]:
###############################
# generate file paths to data #
###############################

hydrated_tweet_folder = "data"
tweet_ids_folder = "data"

path_5g_json = "5g_corona_conspiracy.json"
path_other_json = "other_conspiracy.json"
path_non_consp_json = "non_conspiracy.json"
path_test_json = "test_tweets.json"
path_test_ids_txt = "test_tweet_ids.json"

path_5g = os.path.join(hydrated_tweet_folder, path_5g_json)
path_other = os.path.join(hydrated_tweet_folder, path_other_json)
path_non = os.path.join(hydrated_tweet_folder, path_non_consp_json)
path_test = os.path.join(hydrated_tweet_folder, path_test_json)
path_test_ids = os.path.join(tweet_ids_folder, path_test_ids_txt)

path_lia_data = 'data/FakeNewsChallenge_Features_FirstPrediction.csv'

assert(os.path.isfile(path_5g))
assert(os.path.isfile(path_other))
assert(os.path.isfile(path_non))
assert(os.path.isfile(path_test))
assert(os.path.isfile(path_test_ids))

In [3]:
################
# read in data #
################

fiveg_df = pd.read_json(path_5g)
other_df = pd.read_json(path_other)
nocon_df = pd.read_json(path_non)
test_df = pd.read_json(path_test)

# we will need to submit predictions for all tweet ids
# test_ids_df = pd.read_csv(path_test_ids, names=['id'])
test_ids_df = pd.read_json(path_test_ids)
test_ids_df.rename(columns = {0: 'id'}, inplace = True)

test_id_set = set(test_ids_df['id'])
retreived_test_set = set(test_df['id'])

# find missing tweets from test set
missing_test_tweets = test_id_set.difference(retreived_test_set)

# mark as real tweets, because we're going to add fake tweets later
fiveg_df['actual_tweet'] = True
other_df['actual_tweet'] = True
nocon_df['actual_tweet'] = True

In [4]:
##########################
# read Lia's predictions #
##########################

with open(path_lia_data,'rb') as f:
    contents = StringIO(f.read().decode("iso-8859-1"))
    
lia_df = pd.read_csv(contents, sep=",")

In [5]:
####################
# train eval split #
####################

train_ratio = 0.8

def mark_train(df, train_ratio=0.8, test_ids=None):
    
    if test_ids:
        df['test'] = df.apply(lambda row:(str(row['id']) in test_ids) and row['actual_tweet'], axis=1)
    else:
        df['test'] = df.apply(lambda row: (randint(1,100) > int(train_ratio*100) and row['actual_tweet']), axis=1)            
        
    return df

fiveg_df = mark_train(fiveg_df, train_ratio=train_ratio)
other_df = mark_train(other_df, train_ratio=train_ratio)
nocon_df = mark_train(nocon_df, train_ratio=train_ratio)

In [6]:
####################
# label and concat #
####################

fiveg_df['label'] = 1
other_df['label'] = 2
nocon_df['label'] = 3

print(f"\n{'train':>17} {'test':>12} {'train pct':>15}\n")

def display_ratio(df, name):
    eval_df = df[df['test']==True]
    train_df = df[df['test']==False]
    
    print(f'{name}: {len(train_df):>10,} {len(eval_df):>12,} {len(train_df)/len(df):>14.2f}%')
    
    return train_df, eval_df

fiveg_train_df, _ = display_ratio(fiveg_df, 'FIVEG')
other_train_df, _ = display_ratio(other_df, 'OTHER')
nocon_train_df, _ = display_ratio(nocon_df, 'NOCON')

df = pd.concat([fiveg_df, other_df, nocon_df])

train_df, eval_df = display_ratio(df, 'TOTAL')

X_train = train_df['full_text']
y_train = train_df['label']

X_eval = eval_df['full_text']
y_eval = eval_df['label']


            train         test       train pct

FIVEG:        899          221           0.80%
OTHER:        547          141           0.80%
NOCON:      3,289          849           0.79%
TOTAL:      4,735        1,211           0.80%


In [7]:
no_test = False
if no_test :
    X_train = X_train.append(X_eval)
    y_train = y_train.append(y_eval)

In [8]:
#################
# preprocessing #
#################

class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.re_prog_url = re.compile(r'https://t.co/([a-zA-Z0-9]+)')
    
    def fit( self, X, y=None ):
        return self 
    
    def _process(self, text):
        
        urls = self.re_prog_url.findall(text)
        text = text.lower()\
                .replace('https://t.co/', '')\
                .replace('u.s.', 'us')\
                .replace('u.k.', 'uk')
        for url in urls:
            text = text.replace(url.lower(), 'url')

        return text
    
    def transform(self, X, y=None):
        
        X = X.apply(self._process)
        
        return X

In [10]:
############
# pipeline #
############

classifier = LogisticRegression(
    C=0.9,
    class_weight={
        1: 0.4,
        2: 0.4,
        3: 0.2
    },
    multi_class= 'ovr',
    max_iter=2000,
    solver= 'saga'
)

vectorizer = CountVectorizer(
    strip_accents='unicode',
)

pipeline = Pipeline(
    [
        ('preprocessor', Preprocessor()),
        ('vectorizer', vectorizer),
        ('classifier', classifier)
    ]
)

pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_eval)
probabilities = pipeline.predict_proba(X_eval)

accuracy = accuracy_score(y_eval, predictions)*100
precision = precision_score(y_eval, predictions, zero_division=0, average="macro")*100
recall = recall_score(y_eval, predictions, average="macro")*100
f1 = f1_score(y_eval, predictions, average="macro")*100
support = precision_recall_fscore_support(y_eval, predictions, average="macro")
matthews = matthews_corrcoef(y_eval, predictions)*100

header = classifier.__class__.__name__

print(f'\n{header}\n\nAccuracy  Precision  Recall   F1       MCC')
print(f'{accuracy:.2f}%{precision:>9.2f}%{recall:>10.2f}%{f1:>8.2f}%{matthews:>8.2f}%\n')

##############
# submission #
##############

predictions = pipeline.predict(test_df['full_text'])
probabilities = pipeline.predict_proba(test_df['full_text'])

filename = os.path.join('output','ME20FND_DL-TXST_004.txt')
if no_test:
    filename = os.path.join('output','ME20FND_DL-TXST_004b.txt')

threshold1 = 0.45
threshold2 = 0.15

m = {
    'missing':-1,
    'unk':0,
    'corona_conspiracy':1,
    'other_conspiracy':2,
    'non_conspiracy':3
}

with open(filename,'w') as f:
    for tweet_id, prediction, prob in zip(test_df['id'], predictions, probabilities):
        
        diff = prob[prediction-1]-st.median(prob)
        if diff < threshold1:
            
            z = lia_df[lia_df['tweet_id']==tweet_id]['my_prediction_test_db']
            if len(z) > 0:
                c = m[z.iloc[0]]
                # print(diff, c)
            else:
                c = -1
            
            if c < 1:
                if diff >= threshold2:
                    c = prediction
                else:
                    c = 0
                # print(diff, c)
            
            prediction = c
        
        f.write(f'{tweet_id},{prediction}\n')
    for tweet_id in missing_test_tweets:
        f.write(f'{tweet_id},-1\n')


LogisticRegression

Accuracy  Precision  Recall   F1       MCC
95.87%    94.93%     93.15%   93.94%   91.02%

0.01669878289537896 0
0.01669878289537896 0
0.28621645586985756 0
0.28621645586985756 3
0.24572099355572063 0
0.24572099355572063 3
0.4408717978558876 1
0.3922316694274557 0
0.3922316694274557 1
0.3306098293100742 0
0.3306098293100742 3
0.03360337947782871 1
0.3825084423760857 1
0.24709393674939784 2
0.1946516252521116 2
0.025746628120502435 0
0.025746628120502435 0
0.4453234437042653 0
0.4453234437042653 3
0.13016809184132344 0
0.13016809184132344 0
0.2930083961797586 0
0.2930083961797586 3
0.09778393479077274 2
0.20119481846866172 1
0.05625651703028661 0
0.05625651703028661 0
0.18997336404048037 1
0.4213416365939317 1
0.25762151240398384 1
0.4212138544406947 3
0.43254589009254085 1
0.011459736968115308 0
0.011459736968115308 0
0.3326083241332081 0
0.3326083241332081 3
0.22300200273185838 0
0.22300200273185838 3
0.15412313761677693 0
0.15412313761677693 3
0.27177857600787997 

0.3634947032262982 3
0.06769560056934071 0
0.06769560056934071 0
0.4143325782857507 0
0.4143325782857507 3
0.23847163532772964 1
0.25570798123446253 3
0.23300730817714949 0
0.23300730817714949 3
0.1692535106562253 0
0.1692535106562253 1
0.23766964951890013 0
0.23766964951890013 2
0.10876431516671498 1
0.1555455624849807 0
0.1555455624849807 1
0.28288308457596817 3
0.43655309898192707 1
0.14876036071598964 2
0.05924861547068816 0
0.05924861547068816 0
0.4113767229265367 0
0.4113767229265367 3
0.4111441109297689 0
0.4111441109297689 3
0.36129694009839525 0
0.36129694009839525 1
0.16108842659532296 0
0.16108842659532296 1
0.3190872584457195 0
0.3190872584457195 3
0.302380838286122 2
0.06195695691892705 3
0.261193581921491 0
0.261193581921491 2
0.4216362315032892 1
0.08774890282818498 0
0.08774890282818498 0
0.3408786636596194 0
0.3408786636596194 3
0.2149240822469879 0
0.2149240822469879 1
0.20660696330063094 1
0.40574205428959526 2
0.38303510583769707 0
0.38303510583769707 3
0.1999416756

0.05608131244933978 0
0.05608131244933978 0
0.0034708395568254224 2
0.3981535622289968 2
0.2727070168136565 3
0.2732213100115533 0
0.2732213100115533 1
0.3497386983266489 0
0.3497386983266489 2
0.3869369910250513 0
0.3869369910250513 1
0.22933552923392075 2
0.3996529838884203 2
0.4154966331163636 0
0.4154966331163636 1
0.00010839275858964825 0
0.00010839275858964825 0
0.24648042778521378 1
0.24394793516106855 0
0.24394793516106855 3
0.05408174045888409 0
0.05408174045888409 0
0.41365147705582306 1
0.24194716189420135 0
0.24194716189420135 3
0.36673069425417154 1
0.3761694532578172 0
0.3761694532578172 3
0.4435755799259301 3
0.35921442298430545 1
0.10085002126497855 1
0.19949261684587943 0
0.19949261684587943 3
0.16195548777300633 0
0.16195548777300633 1
0.3452076919347372 3
0.01564620587019011 0
0.01564620587019011 0
0.3310575551559051 0
0.3310575551559051 2
0.052556817348644536 0
0.052556817348644536 0
0.31555607023358917 0
0.31555607023358917 3
0.10454353158807805 1
0.193221803297885

In [ ]:

                                        #####################
                                        # stop here for now #
                                        #####################
            

In [167]:
# m = {1:'FIVEG',2:'OTHER',3:'NOCON'}
m = {0:'OTHER', 1:'FIVEG'}

def print_top100(vectorizer, clf, class_labels):    
    """Prints features with the highest coefficient values, per class"""
    
    feature_names = vectorizer.get_feature_names()
    
    for i, class_label in enumerate(class_labels):
        top100 = np.argsort(clf.coef_[i])[-100:]
        print("%s:\n\n%s\n" % (m[class_label],
              "\n".join(feature_names[j] for j in top100)))
        
print_top100(vectorizer, classifier, [0])

FIVEG:

everywhere
design
map
out
killing
cursing
therefore
harms
28
place
popping
function
whistleblower
spots
loss
alongside
areas
#coronavirus
structure
linked
boss
coverage
testing
discussing
🤦🏽‍♂️
putting
issues
nwo
mentions
flying
created
die
know
@worldtruthtv
aka
turned
project
illness
cv
microwave
frequency
former
chemtrails
ig
connected
cells
version
distancing
kill
#5gtowers
weapon
share
towers
liberty
video
okay
#5gkills
exposure
correlation
london
ain’t
govt
destroy
watch
radiation
21
used
night
side
depopulation
alter
connection
david
#covid
lot
order
play
dying
flu
hemoglobin
city
tell
activating
agenda
emf
electromagnetic
truth
research
link
rolled
body
chris
question
#5g
coincidence
oxygen
immune
5g
symptoms
wuhan

OTHER:

evil
sacrifices
ostensibly
tracked
hold
played
predicted
ppl
chip
confident
ireland
got
eye
kids
right
they're
war
main
digital
censoring
000
frequencies
mike
#qanon
2017
down
lie
steele
studies
funny
getting
military
gates
being
responsible
effects
